# Preprocessing Saham ANTM dengan Faktor Eksternal 
Notebook ini memuat tahapan pembersihan data historis saham ANTM dan integrasi faktor eksternal seperti IHSG, harga emas, dan kurs USD/IDR untuk keperluan prediksi time series menggunakan model CLSTM dan BPNN.

### 🔹 Import library yang diperlukan

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import joblib
import yfinance as yf

### 🔹 Atur direktori kerja ke folder utama proyek

In [12]:
# Atur direktori kerja ke folder utama proyek
os.chdir('D:/ANTMcast')  # bisa juga gunakan '.' jika Anda sudah di root ANTMcast
print("Current Working Directory:", os.getcwd())

Current Working Directory: D:\ANTMcast


### 🔹 Baca data saham ANTM dari folder `data/raw`

In [13]:
csv_path = 'data/raw/ANTM.csv'

if not os.path.exists(csv_path):
    print(f"File '{csv_path}' not found. Pastikan file sudah ada di folder data/raw/.")
else:
    df = pd.read_csv(csv_path)
    display(df.head())

,Date,Price,Open,High,Low,Vol.,Change %
0,06/03/2025,"3,360","3,370","3,390","3,260",278.46M,1.20%
1,06/02/2025,"3,320","3,170","3,330","3,160",324.00M,6.75%
2,05/28/2025,"3,110","3,030","3,220","3,030",285.39M,2.64%
3,05/27/2025,"3,030","3,110","3,120","3,010",193.42M,-2.57%
4,05/26/2025,"3,110","3,170","3,280","3,030",305.57M,-0.64%


### 🔹 Bersihkan kolom, ubah format Volume, dan ubah %Change menjadi float

In [14]:
df.columns = ['Tanggal', 'Price', 'Open', 'High', 'Low', 'Vol', 'Change']
df['Tanggal'] = pd.to_datetime(df['Tanggal'])
df = df.sort_values('Tanggal')
df = df.dropna()

def parse_volume(val):
    val = str(val).replace('M','e6').replace('K','e3')
    try:
        return float(eval(val))
    except:
        return np.nan

df['Volume'] = df['Vol'].apply(parse_volume)
df['%Change'] = df['Change'].str.replace('%','').astype(float)
df = df[['Tanggal', 'Open', 'High', 'Low', 'Price', 'Volume', '%Change']]
df = df.rename(columns={'Price': 'Close'})
df = df.dropna()
df.to_csv('data/processed/antm_cleaned.csv', index=False)
df.head()

,Tanggal,Open,High,Low,Close,Volume,%Change
1548,2019-01-02,770,775,735,740,53130000.0,-3.27
1547,2019-01-03,750,755,740,750,34640000.0,1.35
1546,2019-01-04,750,765,745,760,59880000.0,1.33
1545,2019-01-07,775,785,765,770,65850000.0,1.32
1544,2019-01-08,775,780,755,760,82490000.0,-1.30


❗ *Catatan:* Data eksternal (IHSG, Emas, USD/IDR) akan ditambahkan secara manual jika `yfinance` tidak tersedia. Anda bisa gunakan Google Colab hanya untuk unduh datanya dan simpan ke `data/raw/`, lalu lanjutkan di lokal.

## 🔄 Bersihkan dan Normalisasi Fitur Numerik

- Menghapus koma dari nilai numerik (misalnya '1,025' → 1025).
- Konversi semua fitur numerik menjadi float.
- Normalisasi fitur menggunakan `MinMaxScaler`.
- Simpan hasil dan objek scaler ke folder `data/processed/`.

In [ ]:
df_scaled = df.copy()
fitur = ['Open', 'High', 'Low', 'Close', 'Volume', '%Change']

# Bersihkan koma dan konversi string ke float
for col in fitur:
    df_scaled[col] = df_scaled[col].astype(str).str.replace(',', '').astype(float)

# Normalisasi data
scaler = MinMaxScaler()
df_scaled[fitur] = scaler.fit_transform(df_scaled[fitur])

# Simpan scaler dan hasil
joblib.dump(scaler, 'data/processed/scaler_antm.save')
df_scaled.to_csv('data/processed/antm_scaled.csv', index=False)

df_scaled.head()


ValueError: could not convert string to float: '1,025'

### 🔹 Membuat dataset ber-jendela (windowed) untuk time series modeling

In [ ]:
def create_windowed_dataset(data, window_size=20):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i])
        y.append(data[i, 3])  # kolom ke-3 = 'Close'
    return np.array(X), np.array(y)

dataset = df_scaled[fitur].values
X, y = create_windowed_dataset(dataset)
np.savez_compressed('data/processed/antm_timeseries_window.npz', X=X, y=y)
print("✅ X shape:", X.shape, "| ✅ y shape:", y.shape)

### ✅ Notebook preprocessing selesai
File hasil preprocessing dan normalisasi telah disimpan ke dalam folder `data/processed/`:
- `antm_cleaned.csv`
- `antm_scaled.csv`
- `scaler_antm.save`
- `antm_timeseries_window.npz`